In [ ]:
import pygame
import carla
import random
import math

# Initialize Pygame
pygame.init()

# Screen dimensions
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("CARLA Map Visualization")

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLACK = (0, 0, 0)

# Load the map image (exported from CARLA as PNG/JPEG)
map_image = pygame.image.load("carla_map.png")
map_rect = map_image.get_rect()
map_image.fill()


# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# Get blueprint library and spawn points
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

# Get the vehicle
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# Activate autopilot mode for the vehicle
vehicle.set_autopilot(True)

# Scale and offset for coordinate transformation
MAP_ORIGIN_X, MAP_ORIGIN_Y = -200, -200  # Set this to match your CARLA map bounds
SCALE = 0.1  # Adjust scaling to fit the map in Pygame (lower value to zoom out)

# Camera offset settings (distance above the vehicle)
CAMERA_OFFSET_X = SCREEN_WIDTH // 2
CAMERA_OFFSET_Y = SCREEN_HEIGHT // 2
CAMERA_FOLLOW_SPEED = 0.1  # Controls how smoothly the camera follows the vehicle

# Function to draw an arrow shape for the vehicle
def draw_arrow(surface, color, position, angle, size=10):
    """Draw an arrow at the given position with the given angle."""
    # Arrow points
    arrow_length = size * 3
    arrow_width = size

    # Calculate arrow points
    x, y = position
    angle_rad = math.radians(angle)
    arrow_points = [
        (x + arrow_length * math.cos(angle_rad), y - arrow_length * math.sin(angle_rad)),  # Pointing ahead
        (x + arrow_width * math.cos(angle_rad + math.pi / 2), y - arrow_width * math.sin(angle_rad + math.pi / 2)),  # Right wing
        (x + arrow_width * math.cos(angle_rad - math.pi / 2), y - arrow_width * math.sin(angle_rad - math.pi / 2))  # Left wing
    ]

    pygame.draw.polygon(surface, color, arrow_points)

# Main loop
running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Clear the screen
    screen.fill(WHITE)

    # Get the vehicle's position and rotation (yaw)
    vehicle_transform = vehicle.get_transform()
    carla_x = vehicle_transform.location.x
    carla_y = vehicle_transform.location.y
    vehicle_angle = vehicle_transform.rotation.yaw  # Vehicle rotation in degrees

    # Convert CARLA coordinates to Pygame coordinates
    screen_x = (carla_x - MAP_ORIGIN_X) * SCALE
    screen_y = (carla_y - MAP_ORIGIN_Y) * SCALE

    # Camera follows the vehicle smoothly
    camera_x = CAMERA_OFFSET_X + (screen_x - CAMERA_OFFSET_X) * CAMERA_FOLLOW_SPEED
    camera_y = CAMERA_OFFSET_Y + (screen_y - CAMERA_OFFSET_Y) * CAMERA_FOLLOW_SPEED

    # Create a view that follows the vehicle
    camera_surface = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))
    camera_surface.fill(WHITE)

    # Draw the map on the camera surface
    camera_surface.blit(map_image, map_rect)

    # Draw the vehicle as an arrow
    draw_arrow(camera_surface, RED, (int(screen_x), int(screen_y)), vehicle_angle)

    # Blit the camera surface to the main screen (this simulates the camera effect)
    screen.blit(camera_surface, (0, 0))

    # Update the display
    pygame.display.flip()

    # Cap the frame rate
    clock.tick(30)

# Quit Pygame
pygame.quit()


In [ ]:
import pygame
import carla

# Initialize Pygame
pygame.init()

# Load the map image
MAP_IMAGE_PATH = "carla_map.png"
map_image = pygame.image.load(MAP_IMAGE_PATH)

# Pygame screen dimensions
screen_width, screen_height = map_image.get_width(), map_image.get_height()
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("CARLA Vehicle Tracker")

# CARLA to map scale conversion (adjust as per your map scaling)
CARLA_WORLD_ORIGIN = carla.Location(x=-300, y=-300)  # Adjust this to match your CARLA map's origin
CARLA_WORLD_SCALE = 1.0  # Scale factor to map CARLA coordinates to image pixels

# Map CARLA world location to image coordinates
def carla_to_map(location):
    map_x = int((location.x - CARLA_WORLD_ORIGIN.x) * CARLA_WORLD_SCALE)
    map_y = int((location.y - CARLA_WORLD_ORIGIN.y) * CARLA_WORLD_SCALE)
    return map_x, map_y

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
spawn_points = world.get_map().get_spawn_points()
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle.")
    exit()


# Colors and fonts
vehicle_color = (255, 0, 0)  # Red for the vehicle
font = pygame.font.Font(None, 24)

vehicle.set_autopilot(True)

# Main loop
running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Get vehicle location
    try:
        transform = vehicle.get_transform()
        location = transform.location
        map_x, map_y = carla_to_map(location)
    except RuntimeError:
        print("Failed to get vehicle transform!")
        continue

    # Clear screen and draw map
    screen.blit(map_image, (0, 0))

    # Draw vehicle position on the map
    if 0 <= map_x < screen_width and 0 <= map_y < screen_height:
        pygame.draw.circle(screen, vehicle_color, (map_x, map_y), 5)

    # Display vehicle coordinates
    coord_text = f"Vehicle: ({location.x:.2f}, {location.y:.2f})"
    text_surface = font.render(coord_text, True, (255, 255, 255))
    screen.blit(text_surface, (10, 10))

    # Update display
    pygame.display.flip()
    clock.tick(30)

pygame.quit()


In [ ]:
import pygame
import carla
import math

# Initialize Pygame
pygame.init()

# Load the map image
MAP_IMAGE_PATH = "Town-1.jpg"
map_image = pygame.image.load(MAP_IMAGE_PATH)

# Pygame screen dimensions
screen_width, screen_height = map_image.get_width(), map_image.get_height()
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("CARLA Vehicle Tracker")

# CARLA to map scale conversion (adjust as per your map scaling)
CARLA_WORLD_ORIGIN = carla.Location(x=-300, y=-300)  # Adjust this to match your CARLA map's origin
CARLA_WORLD_SCALE = 1.0  # Scale factor to map CARLA coordinates to image pixels

# Map CARLA world location to image coordinates
def carla_to_map(location):
    map_x = int((location.x - CARLA_WORLD_ORIGIN.x) * CARLA_WORLD_SCALE)
    map_y = int((location.y - CARLA_WORLD_ORIGIN.y) * CARLA_WORLD_SCALE)
    return map_x, map_y

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

# Set synchronous mode for precise updates
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")

# Get spawn points
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available in the map!")
    pygame.quit()
    exit()

# Try to spawn the vehicle
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle. Exiting.")
    pygame.quit()
    exit()

# Attach a spectator camera above the vehicle
spectator = world.get_spectator()

def update_spectator(transform):
    """Position the spectator above the vehicle."""
    location = transform.location
    rotation = transform.rotation
    spectator.set_transform(carla.Transform(
        carla.Location(location.x, location.y, location.z + 50),  # Adjust height
        carla.Rotation(pitch=-90)  # Point the camera downward
    ))

# Vehicle controls
vehicle_control = carla.VehicleControl()
vehicle_control.throttle = 0.0
vehicle_control.steer = 0.0

# Colors and fonts
vehicle_color = (255, 0, 0)  # Red for the vehicle
font = pygame.font.Font(None, 24)

# Main loop
running = True
clock = pygame.time.Clock()

try:
    while running:
        # Process events
        pygame.event.pump()  # Process events manually
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Handle keyboard input for vehicle movement
        keys = pygame.key.get_pressed()
        if keys[pygame.K_8]:
            vehicle_control.throttle = 0.5
        elif keys[pygame.K_2]:
            vehicle_control.brake = 0.5
        else:
            vehicle_control.throttle = 0.0
            vehicle_control.brake = 0.0

        if keys[pygame.K_4]:
            vehicle_control.steer = -0.3
        elif keys[pygame.K_6]:
            vehicle_control.steer = 0.3
        else:
            vehicle_control.steer = 0.0

        # Apply the control to the vehicle
        vehicle.apply_control(vehicle_control)

        # Get vehicle location and update spectator
        transform = vehicle.get_transform()
        location = transform.location
        map_x, map_y = carla_to_map(location)
        update_spectator(transform)

        # Clear screen and draw map
        screen.blit(map_image, (0, 0))

        # Draw vehicle position on the map
        if 0 <= map_x < screen_width and 0 <= map_y < screen_height:
            pygame.draw.circle(screen, vehicle_color, (map_x, map_y), 5)

        # Display vehicle coordinates
        coord_text = f"Vehicle: ({location.x:.2f}, {location.y:.2f})"
        text_surface = font.render(coord_text, True, (255, 255, 255))
        screen.blit(text_surface, (10, 10))

        # Update CARLA world and Pygame display
        world.tick()
        pygame.display.flip()
        clock.tick(30)

finally:
    # Clean up the vehicle to avoid leaving it in CARLA
    if vehicle is not None:
        vehicle.destroy()
    print("Vehicle destroyed. Exiting...")
    pygame.quit()


In [ ]:
# map gen
import carla

client = carla.Client("localhost", 2000)
client.set_timeout(10.0)

world = client.get_world()
map_name = world.get_map().name
xodr_data = world.get_map().to_opendrive()

with open(f"E:\\WindowsNoEditor\\CarlaUE4\\Content\\{map_name}.xodr", "w") as xodr_file:
    xodr_file.write(xodr_data)

print(f"OpenDRIVE map exported as {map_name}.xodr")


In [ ]:
import matplotlib.pyplot as plt
from opendrivepy.xodr_parser import parse_opendrive

# Path to your .xodr file
xodr_file = r"E:\WindowsNoEditor\CarlaUE4\Content\Carla\Maps\OpenDrive\Town10HD.xodr"

# Parse the .xodr file
with open(xodr_file, "r") as file:
    xodr_data = parse_opendrive(file.read())

# Create a plot
plt.figure(figsize=(10, 10))
for road in xodr_data.roads:
    for geometry in road.geometry:
        if hasattr(geometry, 'line') and geometry.line:
            line = geometry.line
            plt.plot([line.start.x, line.end.x], [line.start.y, line.end.y], 'k-')

# Save as PNG
plt.title("OpenDRIVE Road Network")
plt.xlabel("X Coordinates")
plt.ylabel("Y Coordinates")
plt.savefig("output_map.png")
plt.show()


In [ ]:
!pip install opendrive-py

## Carla 2D navigation

In [ ]:
import pygame
import carla
import math

# Initialize Pygame
pygame.init()

# Load the map image
MAP_IMAGE_PATH = "Town-1.jpg"
map_image = pygame.image.load(MAP_IMAGE_PATH)

# Screen dimensions for the zoomed-in map
ZOOM_SCALE = 3  # Adjust for desired zoom level (higher = more zoom)
screen_width, screen_height = 800, 800
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("CARLA Autopilot Tracker")

# CARLA to map scale conversion
CARLA_WORLD_ORIGIN = carla.Location(x=-300, y=-300)  # Adjust for your CARLA map
CARLA_WORLD_SCALE = 1.0  # Conversion factor from CARLA world units to pixels

# Map CARLA world location to image coordinates
def carla_to_map(location):
    map_x = int((location.x - CARLA_WORLD_ORIGIN.x) * CARLA_WORLD_SCALE)
    map_y = int((location.y - CARLA_WORLD_ORIGIN.y) * CARLA_WORLD_SCALE)
    return map_x, map_y

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

# Set synchronous mode for precise updates
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1  # Reduce tick rate for smoother visuals and slower vehicle movement
world.apply_settings(settings)

# Get vehicle blueprint and spawn it
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available!")
    pygame.quit()
    exit()

vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle.")
    pygame.quit()
    exit()

# Attach a spectator camera above the vehicle
spectator = world.get_spectator()

# Autopilot logic: Get waypoints
waypoints = world.get_map().generate_waypoints(distance=5.0)  # Increase distance for smoother path
waypoint_index = 0

def autopilot(vehicle, waypoints, index):
    """Make the vehicle follow waypoints."""
    if index >= len(waypoints):
        return index  # Stop if all waypoints are covered
    target_wp = waypoints[index]
    vehicle.set_transform(target_wp.transform)
    return index + 1

# Colors and fonts
vehicle_color = (255, 0, 0)
arrow_color = (0, 255, 0)
font = pygame.font.Font(None, 24)

# Main loop
running = True
clock = pygame.time.Clock()

try:
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Autopilot: Move the vehicle to the next waypoint
        waypoint_index = autopilot(vehicle, waypoints, waypoint_index)

        # Get vehicle transform and map position
        transform = vehicle.get_transform()
        location = transform.location
        rotation = transform.rotation
        map_x, map_y = carla_to_map(location)

        # Calculate zoomed-in map region
        map_center_x = map_x
        map_center_y = map_y
        crop_rect = pygame.Rect(
            max(0, map_center_x - screen_width // (2 * ZOOM_SCALE)),
            max(0, map_center_y - screen_height // (2 * ZOOM_SCALE)),
            screen_width // ZOOM_SCALE,
            screen_height // ZOOM_SCALE,
        )
        cropped_map = map_image.subsurface(crop_rect).copy()

        # Rotate the map around the center
        rotated_map = pygame.transform.rotate(cropped_map, -rotation.yaw)
        rotated_rect = rotated_map.get_rect(center=(screen_width // 2, screen_height // 2))

        # Draw the rotated map
        screen.fill((0, 0, 0))  # Clear the screen
        screen.blit(rotated_map, rotated_rect.topleft)

        # Draw vehicle arrow at the center of the screen
        arrow_points = [
            (screen_width // 2, screen_height // 2 + 20),
            (screen_width // 2 - 10, screen_height // 2 - 20),
            (screen_width // 2 + 10, screen_height // 2 - 20),
        ]
        pygame.draw.polygon(screen, arrow_color, arrow_points)

        # Display coordinates
        coord_text = f"Vehicle: ({location.x:.2f}, {location.y:.2f})"
        text_surface = font.render(coord_text, True, (255, 255, 255))
        screen.blit(text_surface, (10, 10))

        # Tick the CARLA world and update display
        world.tick()
        pygame.display.flip()
        clock.tick(30)

finally:
    # Clean up
    if vehicle is not None:
        vehicle.destroy()
    print("Vehicle destroyed. Exiting...")
    pygame.quit()


In [ ]:
import pygame
import carla
import math

# Initialize Pygame
pygame.init()

# Load the map image
MAP_IMAGE_PATH = "Town-1.jpg"
map_image = pygame.image.load(MAP_IMAGE_PATH)

# Pygame screen dimensions
screen_width, screen_height = map_image.get_width(), map_image.get_height()
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("CARLA Vehicle Tracker")

# CARLA to map scale conversion
CARLA_WORLD_ORIGIN = carla.Location(x=-300, y=-300)  # Adjust for your CARLA map
CARLA_WORLD_SCALE = 0.9  # Conversion factor from CARLA world units to pixels

# Map CARLA world location to image coordinates
def carla_to_map(location):
    map_x = int((location.x - CARLA_WORLD_ORIGIN.x) * CARLA_WORLD_SCALE)
    map_y = int((location.y - CARLA_WORLD_ORIGIN.y) * CARLA_WORLD_SCALE)
    return map_x, map_y

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

# Set synchronous mode for precise updates
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# Get vehicle blueprint and spawn it
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available!")
    pygame.quit()
    exit()

vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle.")
    pygame.quit()
    exit()

# Attach a spectator camera above and behind the vehicle
spectator = world.get_spectator()

def update_spectator(vehicle_transform):
    """Position the spectator camera to follow the vehicle."""
    location = vehicle_transform.location
    rotation = vehicle_transform.rotation
    spectator.set_transform(
        carla.Transform(
            carla.Location(location.x - 10 * math.cos(math.radians(rotation.yaw)),
                           location.y - 10 * math.sin(math.radians(rotation.yaw)),
                           location.z + 5),  # Height adjustment
            carla.Rotation(pitch=-20, yaw=rotation.yaw)
        )
    )

# Vehicle controls
vehicle_control = carla.VehicleControl()
vehicle_control.throttle = 1 # Adjust throttle for slower movement
vehicle_control.steer = 0.0

# Main loop
running = True
clock = pygame.time.Clock()

try:
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Apply constant forward movement
        vehicle.apply_control(vehicle_control)

        # Get vehicle transform and map position
        transform = vehicle.get_transform()
        location = transform.location
        rotation = transform.rotation
        map_x, map_y = carla_to_map(location)

        # Update the spectator to follow the vehicle
        update_spectator(transform)

        # Clear screen and draw the map
        screen.blit(map_image, (0, 0))

        # Draw the vehicle as an arrow
        if 0 <= map_x < screen_width and 0 <= map_y < screen_height:
            # Arrow shape: Draw rotated arrow to indicate vehicle direction
            arrow_length = 20  # Length of the arrow
            arrow_head = (map_x + arrow_length * math.cos(math.radians(-rotation.yaw)),
                          map_y + arrow_length * math.sin(math.radians(-rotation.yaw)))
            arrow_tail = (map_x, map_y)
            pygame.draw.line(screen, (255, 0, 0), arrow_tail, arrow_head, 3)  # Arrow shaft
            pygame.draw.circle(screen, (255, 0, 0), (int(map_x), int(map_y)), 5)  # Arrow base

        # Update CARLA world and Pygame display
        world.tick()
        pygame.display.flip()
        clock.tick(30)

finally:
    # Clean up
    if vehicle is not None:
        vehicle.destroy()
    print("Vehicle destroyed. Exiting...")
    pygame.quit()


In [ ]:
import pygame
import carla
import math

# Initialize Pygame
pygame.init()

# Load the map image
MAP_IMAGE_PATH = "Town-1.jpg"
map_image = pygame.image.load(MAP_IMAGE_PATH)

# Pygame screen dimensions
screen_width, screen_height = map_image.get_width(), map_image.get_height()
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("CARLA Vehicle Tracker")

# CARLA to map scale conversion
CARLA_WORLD_ORIGIN = carla.Location(x=-300, y=-300)  # Adjust for your CARLA map
CARLA_WORLD_SCALE = 1.2  # Conversion factor from CARLA world units to pixels

# Map CARLA world location to image coordinates
def carla_to_map(location):
    map_x = int((location.x - CARLA_WORLD_ORIGIN.x) * CARLA_WORLD_SCALE)
    map_y = int((location.y - CARLA_WORLD_ORIGIN.y) * CARLA_WORLD_SCALE)
    return map_x, map_y

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

# Set synchronous mode for precise updates
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# Get vehicle blueprint and spawn it
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available!")
    pygame.quit()
    exit()

vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle.")
    pygame.quit()
    exit()

# Attach a spectator camera above and behind the vehicle
spectator = world.get_spectator()

def update_spectator(vehicle_transform):
    """Position the spectator camera to follow the vehicle."""
    location = vehicle_transform.location
    rotation = vehicle_transform.rotation
    spectator.set_transform(
        carla.Transform(
            carla.Location(location.x - 10 * math.cos(math.radians(rotation.yaw)),
                           location.y - 10 * math.sin(math.radians(rotation.yaw)),
                           location.z + 5),  # Height adjustment
            carla.Rotation(pitch=-20, yaw=rotation.yaw)
        )
    )

# Autopilot logic
def get_next_waypoint(vehicle, waypoints):
    """Get the next waypoint for the vehicle to move toward."""
    vehicle_location = vehicle.get_transform().location
    closest_waypoint = min(waypoints, key=lambda wp: vehicle_location.distance(wp.transform.location))
    return closest_waypoint

# Generate waypoints from the map
map = world.get_map()
waypoints = map.generate_waypoints(2.0)  # Distance between waypoints

# Main loop
running = True
clock = pygame.time.Clock()

try:
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Get the next waypoint and set autopilot
        next_waypoint = get_next_waypoint(vehicle, waypoints)
        if next_waypoint:
            vehicle.set_autopilot(True)

        # Get vehicle transform and map position
        transform = vehicle.get_transform()
        location = transform.location
        rotation = transform.rotation
        map_x, map_y = carla_to_map(location)

        # Update the spectator to follow the vehicle
        update_spectator(transform)

        # Clear screen and draw the map
        screen.blit(map_image, (0, 0))

        # Draw the vehicle as an arrow
        if 0 <= map_x < screen_width and 0 <= map_y < screen_height:
            # Arrow shape: Draw rotated arrow to indicate vehicle direction
            arrow_length = 20  # Length of the arrow
            arrow_head = (map_x + arrow_length * math.cos(math.radians(-rotation.yaw)),
                          map_y + arrow_length * math.sin(math.radians(-rotation.yaw)))
            arrow_tail = (map_x, map_y)
            pygame.draw.line(screen, (255, 0, 0), arrow_tail, arrow_head, 3)  # Arrow shaft
            pygame.draw.circle(screen, (255, 0, 0), (int(map_x), int(map_y)), 5)  # Arrow base

        # Update CARLA world and Pygame display
        world.tick()
        pygame.display.flip()
        clock.tick(30)

finally:
    # Clean up
    if vehicle is not None:
        vehicle.destroy()
    print("Vehicle destroyed. Exiting...")
    pygame.quit()


In [ ]:
# 4th attempt ajusting map:

In [ ]:
import pygame
import carla
import math

# Initialize Pygame
pygame.init()

# Load the map image
MAP_IMAGE_PATH = "Town-1.jpg"
map_image = pygame.image.load(MAP_IMAGE_PATH)

# Scale down the map image
ZOOM_FACTOR = 0.3  # Scale factor to reduce map size
map_image = pygame.transform.scale(map_image, (int(map_image.get_width() * ZOOM_FACTOR), int(map_image.get_height() * ZOOM_FACTOR)))

# Pygame screen dimensions
screen_width, screen_height = 800, 600  # Set fixed screen size
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("CARLA Vehicle Tracker")

# CARLA to map scale conversion
CARLA_WORLD_ORIGIN = carla.Location(x=-300, y=-300)  # Adjust for your CARLA map
CARLA_WORLD_SCALE = 0.9 * ZOOM_FACTOR  # Adjust scale factor based on map zoom

# Map CARLA world location to image coordinates
def carla_to_map(location):
    map_x = int((location.x - CARLA_WORLD_ORIGIN.x) * CARLA_WORLD_SCALE)
    map_y = int((location.y - CARLA_WORLD_ORIGIN.y) * CARLA_WORLD_SCALE)
    return map_x, map_y

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

# Set synchronous mode for precise updates
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# Get vehicle blueprint and spawn it
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available!")
    pygame.quit()
    exit()

vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle.")
    pygame.quit()
    exit()

# Attach a spectator camera above and behind the vehicle
spectator = world.get_spectator()

def update_spectator(vehicle_transform):
    """Position the spectator camera to follow the vehicle."""
    location = vehicle_transform.location
    rotation = vehicle_transform.rotation
    spectator.set_transform(
        carla.Transform(
            carla.Location(location.x - 10 * math.cos(math.radians(rotation.yaw)),
                           location.y - 10 * math.sin(math.radians(rotation.yaw)),
                           location.z + 5),  # Height adjustment
            carla.Rotation(pitch=-20, yaw=rotation.yaw)
        )
    )

# Autopilot logic
def get_next_waypoint(vehicle, waypoints):
    """Get the next waypoint for the vehicle to move toward."""
    vehicle_location = vehicle.get_transform().location
    closest_waypoint = min(waypoints, key=lambda wp: vehicle_location.distance(wp.transform.location))
    return closest_waypoint

# Generate waypoints from the map
map = world.get_map()
waypoints = map.generate_waypoints(2.0)  # Distance between waypoints

# Main loop
running = True
clock = pygame.time.Clock()

try:
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Get the next waypoint and set autopilot
        next_waypoint = get_next_waypoint(vehicle, waypoints)
        if next_waypoint:
            vehicle.set_autopilot(True)

        # Get vehicle transform and map position
        transform = vehicle.get_transform()
        location = transform.location
        rotation = transform.rotation
        map_x, map_y = carla_to_map(location)

        # Update the spectator to follow the vehicle
        update_spectator(transform)

        # Clear screen and draw the scaled map centered around the vehicle
        screen.fill((0, 0, 0))
        map_view_x = map_x - screen_width // 2
        map_view_y = map_y - screen_height // 2
        screen.blit(map_image, (-map_view_x, -map_view_y))

        # Draw the vehicle as an arrow in the center of the screen
        arrow_length = 20  # Length of the arrow
        arrow_head = (screen_width // 2 + arrow_length * math.cos(math.radians(-rotation.yaw)),
                      screen_height // 2 + arrow_length * math.sin(math.radians(-rotation.yaw)))
        arrow_tail = (screen_width // 2, screen_height // 2)
        pygame.draw.line(screen, (255, 0, 0), arrow_tail, arrow_head, 3)  # Arrow shaft
        pygame.draw.circle(screen, (255, 0, 0), (screen_width // 2, screen_height // 2), 5)  # Arrow base

        # Update CARLA world and Pygame display
        world.tick()
        pygame.display.flip()
        clock.tick(30)

finally:
    # Clean up
    if vehicle is not None:
        vehicle.destroy()
    print("Vehicle destroyed. Exiting...")
    pygame.quit()


In [ ]:
import pygame
import carla
import math

# Initialize Pygame
pygame.init()

# Load the map image
MAP_IMAGE_PATH = "Town-1.jpg"
map_image = pygame.image.load(MAP_IMAGE_PATH)
map_width, map_height = map_image.get_width(), map_image.get_height()
screen = pygame.display.set_mode((map_width, map_height))
pygame.display.set_caption("CARLA Vehicle Tracker")

# CARLA to map scale conversion (adjusted to match 679x670 dimensions)
CARLA_WORLD_ORIGIN = carla.Location(x=-300, y=-300)  # Adjust CARLA map origin
CARLA_WORLD_SCALE_X = map_width / 600.0  # Assume CARLA world width ~600 units
CARLA_WORLD_SCALE_Y = map_height / 600.0  # Assume CARLA world height ~600 units

def carla_to_map(location):
    """Convert CARLA world location to map image coordinates."""
    map_x = int((location.x - CARLA_WORLD_ORIGIN.x) * CARLA_WORLD_SCALE_X)
    map_y = int((location.y - CARLA_WORLD_ORIGIN.y) * CARLA_WORLD_SCALE_Y)
    return map_x, map_y

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

# Set synchronous mode for precise updates
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# Get vehicle blueprint and spawn it
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available!")
    pygame.quit()
    exit()

vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle.")
    pygame.quit()
    exit()

# Attach a spectator camera above and behind the vehicle
spectator = world.get_spectator()

def update_spectator(vehicle_transform):
    """Position the spectator camera to follow the vehicle."""
    location = vehicle_transform.location
    rotation = vehicle_transform.rotation
    spectator.set_transform(
        carla.Transform(
            carla.Location(location.x - 10 * math.cos(math.radians(rotation.yaw)),
                           location.y - 10 * math.sin(math.radians(rotation.yaw)),
                           location.z + 5),  # Height adjustment
            carla.Rotation(pitch=-20, yaw=rotation.yaw)
        )
    )

# Autopilot logic
def get_next_waypoint(vehicle, waypoints):
    """Get the next waypoint for the vehicle to move toward."""
    vehicle_location = vehicle.get_transform().location
    closest_waypoint = min(waypoints, key=lambda wp: vehicle_location.distance(wp.transform.location))
    return closest_waypoint

# Generate waypoints from the map
map = world.get_map()
waypoints = map.generate_waypoints(2.0)  # Distance between waypoints

# Main loop
running = True
clock = pygame.time.Clock()

try:
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Get the next waypoint and set autopilot
        next_waypoint = get_next_waypoint(vehicle, waypoints)
        if next_waypoint:
            vehicle.set_autopilot(True)

        # Get vehicle transform and map position
        transform = vehicle.get_transform()
        location = transform.location
        rotation = transform.rotation
        map_x, map_y = carla_to_map(location)

        # Update the spectator to follow the vehicle
        update_spectator(transform)

        # Clear screen and draw the map
        screen.fill((0, 0, 0))  # Clear screen
        screen.blit(map_image, (0, 0))

        # Draw the vehicle as an arrow
        if 0 <= map_x < map_width and 0 <= map_y < map_height:
            # Arrow shape: Draw rotated arrow to indicate vehicle direction
            arrow_length = 20  # Length of the arrow
            arrow_head = (map_x + arrow_length * math.cos(math.radians(-rotation.yaw)),
                          map_y + arrow_length * math.sin(math.radians(-rotation.yaw)))
            arrow_tail = (map_x, map_y)
            pygame.draw.line(screen, (255, 0, 0), arrow_tail, arrow_head, 3)  # Arrow shaft
            pygame.draw.circle(screen, (255, 0, 0), (int(map_x), int(map_y)), 5)  # Arrow base

        # Update CARLA world and Pygame display
        world.tick()
        pygame.display.flip()
        clock.tick(30)

finally:
    # Clean up
    if vehicle is not None:
        vehicle.destroy()
    print("Vehicle destroyed. Exiting...")
    pygame.quit()


In [ ]:
import pygame
import carla
import math

# Initialize Pygame
pygame.init()

# Load the map image
MAP_IMAGE_PATH = "Town-1.jpg"
map_image = pygame.image.load(MAP_IMAGE_PATH)
map_width, map_height = map_image.get_width(), map_image.get_height()
screen = pygame.display.set_mode((map_width, map_height))
pygame.display.set_caption("CARLA Vehicle Tracker")

# CARLA to map scale conversion (adjusted to match 679x670 dimensions)
CARLA_WORLD_ORIGIN = carla.Location(x=-300, y=-300)  # Adjust CARLA map origin
CARLA_WORLD_SCALE_X = map_width / 600.0  # Assume CARLA world width ~600 units
CARLA_WORLD_SCALE_Y = map_height / 600.0  # Assume CARLA world height ~600 units

# Limit CARLA coordinates to fit within the map image
CARLA_MIN_X, CARLA_MAX_X = -300, 300
CARLA_MIN_Y, CARLA_MAX_Y = -300, 300

def carla_to_map(location):
    """Convert CARLA world location to map image coordinates, clamping to valid bounds."""
    clamped_x = max(CARLA_MIN_X, min(CARLA_MAX_X, location.x))
    clamped_y = max(CARLA_MIN_Y, min(CARLA_MAX_Y, location.y))
    map_x = int((clamped_x - CARLA_WORLD_ORIGIN.x) * CARLA_WORLD_SCALE_X)
    map_y = int((clamped_y - CARLA_WORLD_ORIGIN.y) * CARLA_WORLD_SCALE_Y)
    return map_x, map_y

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

# Set synchronous mode for precise updates
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# Get vehicle blueprint and spawn it
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available!")
    pygame.quit()
    exit()

vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle.")
    pygame.quit()
    exit()

# Attach a spectator camera above and behind the vehicle
spectator = world.get_spectator()

def update_spectator(vehicle_transform):
    """Position the spectator camera to follow the vehicle."""
    location = vehicle_transform.location
    rotation = vehicle_transform.rotation
    spectator.set_transform(
        carla.Transform(
            carla.Location(location.x - 10 * math.cos(math.radians(rotation.yaw)),
                           location.y - 10 * math.sin(math.radians(rotation.yaw)),
                           location.z + 5),  # Height adjustment
            carla.Rotation(pitch=-20, yaw=rotation.yaw)
        )
    )

# Autopilot logic
def get_next_waypoint(vehicle, waypoints):
    """Get the next waypoint for the vehicle to move toward."""
    vehicle_location = vehicle.get_transform().location
    closest_waypoint = min(waypoints, key=lambda wp: vehicle_location.distance(wp.transform.location))
    return closest_waypoint

# Generate waypoints from the map
map = world.get_map()
waypoints = map.generate_waypoints(2.0)  # Distance between waypoints

# Main loop
running = True
clock = pygame.time.Clock()

try:
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Get the next waypoint and set autopilot
        next_waypoint = get_next_waypoint(vehicle, waypoints)
        if next_waypoint:
            vehicle.set_autopilot(True)

        # Get vehicle transform and map position
        transform = vehicle.get_transform()
        location = transform.location
        rotation = transform.rotation
        map_x, map_y = carla_to_map(location)

        # Update the spectator to follow the vehicle
        update_spectator(transform)

        # Clear screen and draw the map
        screen.fill((0, 0, 0))  # Clear screen
        screen.blit(map_image, (0, 0))

        # Draw the vehicle as an arrow
        if 0 <= map_x < map_width and 0 <= map_y < map_height:
            # Arrow shape: Draw rotated arrow to indicate vehicle direction
            arrow_length = 20  # Length of the arrow
            arrow_head = (map_x + arrow_length * math.cos(math.radians(-rotation.yaw)),
                          map_y + arrow_length * math.sin(math.radians(-rotation.yaw)))
            arrow_tail = (map_x, map_y)
            pygame.draw.line(screen, (255, 0, 0), arrow_tail, arrow_head, 3)  # Arrow shaft
            pygame.draw.circle(screen, (255, 0, 0), (int(map_x), int(map_y)), 5)  # Arrow base

        # Update CARLA world and Pygame display
        world.tick()
        pygame.display.flip()
        clock.tick(30)

finally:
    # Clean up
    if vehicle is not None:
        vehicle.destroy()
    print("Vehicle destroyed. Exiting...")
    pygame.quit()


### position controoler in map


In [ ]:
import pygame
import carla
import math

# Initialize Pygame
pygame.init()

# Load the map image
MAP_IMAGE_PATH = "navigator.png"
map_image = pygame.image.load(MAP_IMAGE_PATH)
map_width, map_height = map_image.get_width(), map_image.get_height()
screen = pygame.display.set_mode((map_width, map_height))
screen.fill("#535550")
pygame.display.set_caption("CARLA Vehicle Tracker")

# CARLA to map scale conversion (adjusted to match 679x670 dimensions)
CARLA_WORLD_ORIGIN = carla.Location(x=-300, y=-300)  # Adjust CARLA map origin
CARLA_WORLD_SCALE_X = map_width / 600.0  # Assume CARLA world width ~600 units
CARLA_WORLD_SCALE_Y = map_height / 600.0  # Assume CARLA world height ~600 units

# Limit CARLA coordinates to fit within the map image
CARLA_MIN_X, CARLA_MAX_X = -300, 300
CARLA_MIN_Y, CARLA_MAX_Y = -300, 300

def carla_to_map(location):
    """Convert CARLA world location to map image coordinates, clamping to valid bounds."""
    clamped_x = max(CARLA_MIN_X, min(CARLA_MAX_X, location.x))
    clamped_y = max(CARLA_MIN_Y, min(CARLA_MAX_Y, location.y))
    map_x = int((clamped_x - CARLA_WORLD_ORIGIN.x) * CARLA_WORLD_SCALE_X)
    map_y = int((clamped_y - CARLA_WORLD_ORIGIN.y) * CARLA_WORLD_SCALE_Y)
    return map_x, map_y

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

# Set synchronous mode for precise updates
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# Get vehicle blueprint and spawn it
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available!")
    pygame.quit()
    exit()

vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle.")
    pygame.quit()
    exit()

# Attach a spectator camera above and behind the vehicle
spectator = world.get_spectator()

def update_spectator(vehicle_transform):
    """Position the spectator camera to follow the vehicle."""
    location = vehicle_transform.location
    rotation = vehicle_transform.rotation
    spectator.set_transform(
        carla.Transform(
            carla.Location(location.x - 10 * math.cos(math.radians(rotation.yaw)),
                           location.y - 10 * math.sin(math.radians(rotation.yaw)),
                           location.z + 5),  # Height adjustment
            carla.Rotation(pitch=-20, yaw=rotation.yaw)
        )
    )

# Control menu state variables
map_scale = 1.0  # Default map scale
map_offset_x, map_offset_y = 0, 0  # Default map position offset
map_rotation = 0  # Default map rotation

# Autopilot logic
def get_next_waypoint(vehicle, waypoints):
    """Get the next waypoint for the vehicle to move toward."""
    vehicle_location = vehicle.get_transform().location
    closest_waypoint = min(waypoints, key=lambda wp: vehicle_location.distance(wp.transform.location))
    return closest_waypoint

# Generate waypoints from the map
map = world.get_map()
waypoints = map.generate_waypoints(2.0)  # Distance between waypoints

# Main loop
running = True
clock = pygame.time.Clock()

try:
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            # Handle control inputs
            keys = pygame.key.get_pressed()
            if keys[pygame.K_UP]:
                map_offset_y -= 5
            if keys[pygame.K_DOWN]:
                map_offset_y += 5
            if keys[pygame.K_LEFT]:
                map_offset_x -= 5
            if keys[pygame.K_RIGHT]:
                map_offset_x += 5
            if keys[pygame.K_w]:
                map_scale += 0.01
            if keys[pygame.K_s]:
                map_scale -= 0.01
            if keys[pygame.K_a]:
                map_rotation -= 1
            if keys[pygame.K_d]:
                map_rotation += 1

        # Get the next waypoint and set autopilot
        next_waypoint = get_next_waypoint(vehicle, waypoints)
        if next_waypoint:
            vehicle.set_autopilot(True)

        # Get vehicle transform and map position
        transform = vehicle.get_transform()
        location = transform.location
        rotation = transform.rotation
        map_x, map_y = carla_to_map(location)

        # Update the spectator to follow the vehicle
        update_spectator(transform)

        # Clear screen
        screen.fill((0, 0, 0))

        # Transform and draw the map
        scaled_width = int(map_width * map_scale)
        scaled_height = int(map_height * map_scale)
        scaled_map = pygame.transform.scale(map_image, (scaled_width, scaled_height))
        rotated_map = pygame.transform.rotate(scaled_map, map_rotation)
        screen.blit(rotated_map, (map_offset_x, map_offset_y))

        # Draw the vehicle as an arrow
        if 0 <= map_x < map_width and 0 <= map_y < map_height:
            arrow_length = 20  # Length of the arrow
            arrow_head = (map_x + arrow_length * math.cos(math.radians(-rotation.yaw)),
                          map_y + arrow_length * math.sin(math.radians(-rotation.yaw)))
            arrow_tail = (map_x, map_y)
            pygame.draw.line(screen, (255, 0, 0), arrow_tail, arrow_head, 3)
            pygame.draw.circle(screen, (255, 0, 0), (int(map_x), int(map_y)), 5)

        # Update CARLA world and Pygame display
        world.tick()
        pygame.display.flip()
        clock.tick(30)

finally:
    # Clean up
    if vehicle is not None:
        vehicle.destroy()
    print("Vehicle destroyed. Exiting...")
    pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import carla

try:
    client = carla.Client("localhost", 2000)
    client.set_timeout(10.0)
    world = client.get_world()
    print("Connected to CARLA!")
except Exception as e:
    print(f"Failed to connect to CARLA: {e}")
